In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
from sklearn.impute import SimpleImputer

In [5]:
simr = SimpleImputer(missing_values=np.nan, strategy='median')

In [6]:
train.Age.isnull().sum()

177

In [7]:
train_num = train.drop(["Name","Sex","Ticket","Cabin","Embarked"], axis=1)

In [8]:
train_num.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.0,1,0,7.2500
1,2,1,1,38.0,1,0,71.2833
2,3,1,3,26.0,0,0,7.9250
3,4,1,1,35.0,1,0,53.1000
4,5,0,3,35.0,0,0,8.0500


In [9]:
simr.fit(train_num)

SimpleImputer(strategy='median')

In [10]:
simr.statistics_

array([446.    ,   0.    ,   3.    ,  28.    ,   0.    ,   0.    ,
        14.4542])

In [11]:
train_num.median().values

array([446.    ,   0.    ,   3.    ,  28.    ,   0.    ,   0.    ,
        14.4542])

In [12]:
simr.fit_transform(train_num)

array([[  1.    ,   0.    ,   3.    , ...,   1.    ,   0.    ,   7.25  ],
       [  2.    ,   1.    ,   1.    , ...,   1.    ,   0.    ,  71.2833],
       [  3.    ,   1.    ,   3.    , ...,   0.    ,   0.    ,   7.925 ],
       ...,
       [889.    ,   0.    ,   3.    , ...,   1.    ,   2.    ,  23.45  ],
       [890.    ,   1.    ,   1.    , ...,   0.    ,   0.    ,  30.    ],
       [891.    ,   0.    ,   3.    , ...,   0.    ,   0.    ,   7.75  ]])

In [13]:
simr.statistics_

array([446.    ,   0.    ,   3.    ,  28.    ,   0.    ,   0.    ,
        14.4542])

In [14]:
X = simr.fit_transform(train_num)

In [15]:
train_tr = pd.DataFrame(X, columns=train_num.columns, index = list(train.index.values))

In [16]:
train_tr

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1.0,0.0,3.0,22.0,1.0,0.0,7.2500
1,2.0,1.0,1.0,38.0,1.0,0.0,71.2833
2,3.0,1.0,3.0,26.0,0.0,0.0,7.9250
3,4.0,1.0,1.0,35.0,1.0,0.0,53.1000
4,5.0,0.0,3.0,35.0,0.0,0.0,8.0500
...,...,...,...,...,...,...,...
886,887.0,0.0,2.0,27.0,0.0,0.0,13.0000
887,888.0,1.0,1.0,19.0,0.0,0.0,30.0000
888,889.0,0.0,3.0,28.0,1.0,2.0,23.4500
889,890.0,1.0,1.0,26.0,0.0,0.0,30.0000


In [17]:
train_tr.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
dtype: int64

In [18]:
train_cat = train[['Sex','Embarked']]

In [19]:
train_cat.head()

,Sex,Embarked
0,male,S
1,female,C
2,female,S
3,female,S
4,male,S


In [20]:
train_sex = train['Sex']
train_emb = train['Embarked']

In [21]:
train_sex_encoded, train_sex_categories = train_sex.factorize()
train_emb_encoded, train_emb_categories = train_emb.factorize()

In [22]:
train_emb_encoded[:10]

array([0, 1, 0, 0, 0, 2, 0, 0, 0, 1], dtype=int64)

In [23]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

In [24]:
train_sex_1hot = encoder.fit_transform(train_sex_encoded.reshape(-1,1))
train_emb_1hot = encoder.fit_transform(train_emb_encoded.reshape(-1,1))

In [25]:
train_sex_1hot.toarray()

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [26]:
train_emb_1hot.toarray()

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [27]:
list(train_num)

['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [28]:
from sklearn.base import BaseEstimator, TransformerMixin

sibsp_ix, parch_ix = 4,5

class CombinedAttributesAdder(BaseEstimator, TransformerMixin) :

    def __init__(self, add_family_size=True) :
        self.add_family_size = add_family_size

    def fit(self, X, y=None) :
        return self

    def transform(self, X, y=None) :
        family_size = X[:,sibsp_ix] + X[:,parch_ix] + 1
        return np.c_[np.delete(X,[4,5],axis=1),family_size]

In [29]:
attr_adder = CombinedAttributesAdder(add_family_size=True)

In [30]:
train_extra_attribs = attr_adder.transform(train_num.values)

In [31]:
train_extra_attribs

array([[  1.    ,   0.    ,   3.    ,  22.    ,   7.25  ,   2.    ],
       [  2.    ,   1.    ,   1.    ,  38.    ,  71.2833,   2.    ],
       [  3.    ,   1.    ,   3.    ,  26.    ,   7.925 ,   1.    ],
       ...,
       [889.    ,   0.    ,   3.    ,      nan,  23.45  ,   4.    ],
       [890.    ,   1.    ,   1.    ,  26.    ,  30.    ,   1.    ],
       [891.    ,   0.    ,   3.    ,  32.    ,   7.75  ,   1.    ]])

In [32]:
train_tr = pd.DataFrame(train_extra_attribs)

In [33]:
train_tr = pd.DataFrame(train_extra_attribs, columns=['PassengerId', 'Survived', 'Pclass', 'Age', 'Fare','FamilySize'], index= list(train.index.values))

In [34]:
train_tr

,PassengerId,Survived,Pclass,Age,Fare,FamilySize
0,1.0,0.0,3.0,22.0,7.2500,2.0
1,2.0,1.0,1.0,38.0,71.2833,2.0
2,3.0,1.0,3.0,26.0,7.9250,1.0
3,4.0,1.0,1.0,35.0,53.1000,2.0
4,5.0,0.0,3.0,35.0,8.0500,1.0
...,...,...,...,...,...,...
886,887.0,0.0,2.0,27.0,13.0000,1.0
887,888.0,1.0,1.0,19.0,30.0000,1.0
888,889.0,0.0,3.0,NaN,23.4500,4.0
889,890.0,1.0,1.0,26.0,30.0000,1.0


In [35]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin) :

    def __init__(self, attribute_names):
        self.attribute_names = attribute_names

    def fit(self, X, y=None) :
        return self

    def transform(self, X) :
        return X[self.attribute_names].values

In [36]:
class CategoricalEncoder(BaseEstimator, TransformerMixin) :

    def __init__(self, to_categorical_onehot) :
        self.to_categorical_onehot = to_categorical_onehot

    def fit(self, X, y=None) :
        return self

    def transform(self, X, y=None) :
        X_encoded, X_categories = X.factorize()
        encoder = OneHotEncoder()

        return encoder.fit_transform(X_encoded.reshape(-1,1))

In [37]:
cat_encoder = CategoricalEncoder(to_categorical_onehot=True)

In [38]:
train_sex_cat = cat_encoder.transform(train['Sex'])

In [39]:
train_sex_cat

<891x2 sparse matrix of type '<class 'numpy.float64'>'
	with 891 stored elements in Compressed Sparse Row format>

In [40]:
train_sex_cat.toarray()

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_attribs = list(train_num)
cat_attribs = ["Sex","Embarked"]

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('simple_imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('selector',DataFrameSelector(cat_attribs)),
    ('cat_endocer',CategoricalEncoder(to_categorical_onehot=True))
])

In [42]:
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline)
])

In [43]:
class CategoricalEncoder(BaseEstimator, TransformerMixin) :

    def __init__(self, to_categorical_onehot) :
        self.to_categorical_onehot = to_categorical_onehot

    def fit(self, X, y=None) :
        return self

    def transform(self, X, y=None) :
        
        encoder = OneHotEncoder()
        for i,cat in enumerate(cat_attrbs) :
        
            X_encoded, X_categories = X[:,i].factorize()
            X_cated[:,i] = encoder.fit_transform(X_encoded.reshape(-1,1))

        return X_cated

In [44]:
train[cat_attribs].values

array([['male', 'S'],
       ['female', 'C'],
       ['female', 'S'],
       ...,
       ['female', 'S'],
       ['male', 'C'],
       ['male', 'Q']], dtype=object)

In [45]:
train[cat_attribs].values[:,0]

array(['male', 'female', 'female', 'female', 'male', 'male', 'male',
       'male', 'female', 'female', 'female', 'female', 'male', 'male',
       'female', 'female', 'male', 'male', 'female', 'female', 'male',
       'male', 'female', 'male', 'female', 'female', 'male', 'male',
       'female', 'male', 'male', 'female', 'female', 'male', 'male',
       'male', 'male', 'male', 'female', 'female', 'female', 'female',
       'male', 'female', 'female', 'male', 'male', 'female', 'male',
       'female', 'male', 'male', 'female', 'female', 'male', 'male',
       'female', 'male', 'female', 'male', 'male', 'female', 'male',
       'male', 'male', 'male', 'female', 'male', 'female', 'male', 'male',
       'female', 'male', 'male', 'male', 'male', 'male', 'male', 'male',
       'female', 'male', 'male', 'female', 'male', 'female', 'female',
       'male', 'male', 'female', 'male', 'male', 'male', 'male', 'male',
       'male', 'male', 'male', 'male', 'female', 'male', 'female', 'male',
      

In [46]:
type(train['Sex'])

pandas.core.series.Series

In [47]:
pd.Series(train[cat_attribs].values[:,0]).factorize()

(array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
        1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
        1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 

In [48]:
class CategoricalEncoder(BaseEstimator, TransformerMixin) :

    def __init__(self, to_categorical_onehot) :
        self.to_categorical_onehot = to_categorical_onehot

    def fit(self, X, y=None) :
        return self

    def transform(self, X, y=None) :
        
        encoder = OneHotEncoder()
        for i,cat in enumerate(cat_attribs) :
        
            X_encoded, X_categories = (pd.Series(X[:,i])).factorize()
            X_cated[:,i] = encoder.fit_transform(X_encoded.reshape(-1,1))

        return X_cated

In [49]:
catencoder = CategoricalEncoder(to_categorical_onehot=True)

In [50]:
for i,cat in enumerate(cat_attribs) :

    X_encoded, X_categories = (pd.Series(train[cat_attribs].values[:,i])).factorize()

In [51]:
test_data = pd.read_csv('test.csv')

In [52]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
